## Урок 4. Тематическое моделирование. EM-алгоритм/ ДЗ

продолжаем исследование датасета с твитами

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук `parse_rospotrebnadzor.ipynb`
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [7]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

--2023-12-23 14:02:10--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2023-12-23 14:02:10--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc186828419590b8a5637563e279.dl.dropboxusercontent.com/cd/0/inline/CJ-Z4h2gtpRRpkSdwXqelxMFmFbNs_G0CntP-luhPrU-w3aKgq2AwotDD97_nN8CaVajmapF1W3XB3uNTAuZ9H3mHbavo_dg85o2L18MxLStPdaQJbHkpxUxL5TLspOY8JY/file# [following]
--2023-12-23 14:02:11--  https://uc186828419590b8a5637563e279.dl.dropboxusercontent.com/cd/0/inline/CJ-Z4h2gtpRRpkSdwXqelxMFmFbNs_G0CntP-luhPrU-w3aKgq2AwotDD97_nN8CaVajmapF1W3XB3uNTAuZ9H3mHbavo_dg85o2L18MxLStPdaQJbHkpxUxL5

In [8]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2023-12-23 14:02:11--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2023-12-23 14:02:11--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3b00255d0f6df1687d2a799674.dl.dropboxusercontent.com/cd/0/inline/CJ-0PmUd0LABIcwmb07bwz3Mkr1jVXS1-sQ4v-jYYLix1QmS0YSBwi2QiCwE0ZWhm_6YwylqkknEFq6dxrFLyw-2WPuNm2exhxUXn84_P7ag6Zz8jcN7mo-rUtIbJbn7bCY/file# [following]
--2023-12-23 14:02:12--  https://uc3b00255d0f6df1687d2a799674.dl.dropboxusercontent.com/cd/0/inline/CJ-0PmUd0LABIcwmb07bwz3Mkr1jVXS1-sQ4v-jYYLix1QmS0YSBwi2QiCwE0ZWhm_6YwylqkknEFq6dxrFLyw-2WPuNm2exhxUXn84_P7ag6Zz8jcN7mo-rUt

In [10]:
!pip install pymorphy2 pyLDAvis

In [11]:
!pip install -U gensim

In [12]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=1c5f3a92de5a3c186fb2aa0d2ca462e97e4c5260e061ab4515d8808bb513192a
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [13]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [14]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
import nltk

import pymorphy2
from gensim.models import *
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [15]:
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

In [ ]:
morpher = MorphAnalyzer()

In [ ]:
stopwords_list = set(get_stop_words("ru"))

In [ ]:
positive = pd.read_csv('positive.csv', sep = ';', usecols = [3], names = ['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep = ';', usecols = [3], names = ['text'])
negative['label'] = ['negative'] * len(negative)

In [24]:
df = pd.concat([positive, negative], ignore_index = True)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
words_regex = re.compile(r'\w+')

# Формирование токенов длиной более 3 букв
def find_words(text, regex = words_regex):
    tokens = regex.findall(text)
    return [w for w in tokens if w.isalpha() and len(w)>= 4]


In [ ]:
# Удаляем стоп слова и другие символы, не являющимися буквами
def lemmatize(words, lemmer = morpher, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords and w.isalpha()]

#Чистим от нецензурных слов и слов, не несущих особой смысловой нагрузки
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'значит', ' ',  string_tokens)
    string_tokens = re.sub(r'почему', ' ',  string_tokens)
    string_tokens = re.sub(r'свой', ' ',  string_tokens)
    string_tokens = re.sub(r'капец', ' ',  string_tokens)
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

In [28]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


In [ ]:
# Словарь уникальных токенов
dictionary = corpora.Dictionary(df['text'])

In [31]:
# Удаление редких слов
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')
len(dictionary.keys())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


8100

In [ ]:
# Векторизация твитов
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [38]:
# Обработка словаря
mm = corpora.MmCorpus('tweet.model')
print(df['text'].values[0])
print(mm[0])

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# Обучение модели
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [40]:
# Определение тематики
lda.show_topics(num_topics=3, num_words=20, formatted=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.020*"дом" + 0.018*"новый" + 0.015*"никто" + 0.014*"жаль" + 0.014*"мама" + 0.013*"школа" + 0.012*"настроение" + 0.011*"урок" + 0.011*"работа" + 0.010*"идти" + 0.010*"прийти" + 0.009*"экзамен" + 0.009*"скоро" + 0.008*"бояться" + 0.007*"неделя" + 0.007*"улица" + 0.006*"чувство" + 0.006*"парень" + 0.006*"хороший" + 0.006*"твит"'),
 (1,
  '0.033*"завтра" + 0.019*"больш" + 0.014*"хороший" + 0.014*"утро" + 0.012*"друг" + 0.009*"школа" + 0.008*"вчера" + 0.007*"прийтись" + 0.007*"нравиться" + 0.007*"фильм" + 0.007*"закончиться" + 0.007*"вроде" + 0.007*"боль" + 0.006*"голова" + 0.006*"снег" + 0.006*"вечер" + 0.006*"ужасный" + 0.006*"фотка" + 0.006*"дом" + 0.006*"учёба"'),
 (2,
  '0.014*"час" + 0.012*"любимый" + 0.012*"ночь" + 0.010*"остаться" + 0.010*"последний" + 0.009*"каникулы" + 0.009*"дело" + 0.009*"казаться" + 0.009*"жалко" + 0.008*"самый" + 0.008*"телефон" + 0.008*"правда" + 0.007*"слово" + 0.006*"месяц" + 0.006*"конец" + 0.006*"пара" + 0.006*"момент" + 0.006*"выйти" + 0.006*"уж

In [41]:
# Оценка качества модели
print(lda.log_perplexity(corpus))

-8.13572294610012


In [42]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.00029288719523534317


In [43]:
coherence_model_lda = CoherenceModel(model=lda, texts=df['text'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

Средняя когерентность:  0.2508417445135038


In [44]:
visual_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(visual_data)

In [ ]:
pyLDAvis.save_html(visual_data, 'HW_04_visual.html')

[ссылка на файл визуализации](https://drive.google.com/file/d/1o6sdmS07sDn8oX9nXC4Sk8bjCrCJ-Te-/view?usp=drive_link)

### Основные темы:
* учёба
* работа
* отдых
* взаимотношения  